In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np

import geone
import geone.imgplot as imgplt
import geone.customcolors as ccol
import geone.deesseinterface as dsi
from geone import grf
import geone.covModel as gcm
import geone.imgplot as imgplt
import geone.img as img

import pybert as pb
import pygimli as pg
import pygimli.meshtools as mt  # save space
from matplotlib import colors


ModuleNotFoundError: No module named 'geone'

# Useless for the moment

In [ ]:
# TI
ti = geone.img.readImageGslib('lagoon3.gslib')
geone.imgplot.drawImage2D(ti, categ=True)

In [ ]:
model_Pop = ti.val[0,0,:,:]
Grid_str = mt.grid.createGrid(np.where(model_Pop == 1)[1],- np.where(model_Pop == 1)[0],marker = 2,worldBoundaryMarker =True )
type(Grid_str)
pg.show(Grid_str)

In [ ]:
nz = model_Pop.shape[0]
nx = model_Pop.shape[1]

right_side = np.argmax(model_Pop,axis=1)
left_side = nx - np.argmax(np.fliplr(model_Pop),axis=1)

res_map = model_Pop
res_map[res_map == 0] = 20
res_map[res_map == 1] = 50
res_map[0:15,:]= 100


#res_map[25:27,:]= 50

plt.imshow(res_map,norm=colors.LogNorm())
plt.gca().invert_yaxis()
plt.title('Apparent Rho')
plt.ylabel('Dipole Distance')
plt.xlabel('MidPoint')
plt.colorbar()

In [ ]:
world = mt.createWorld(start=[0, 0], end=[nx, -nz],
                       worldMarker=True,marker=1)


# Merge geometry definition into a Piecewise Linear Complex (PLC)
geom = mt.mergePLC([world])

# Optional: show the geometry
pg.show(geom)

In [ ]:
scheme = pb.createData(elecs=pg.utils.grange(start=0, end=200, n=72),
                       schemeName='dd')
for pos in scheme.sensorPositions():
    geom.createNode(pos)
    geom.createNode(pos + pg.RVector3(0, -0.1))

In [ ]:
mesh = mt.createMesh(geom, quality=34)

# Create a map to set resistivity values in the appropriate regions
# [[regionNumber, resistivity], [regionNumber, resistivity], [...]

# Optional: take a look at the mesh
pg.show(mesh, label='Resistivity $(\Omega$m)', showMesh=True)


In [ ]:
cell_centers = np.array(mesh.cellCenters())
rho_cells = res_map[np.round(cell_centers[:,1] + nz - 1).astype(int),np.round(cell_centers[:,0] - 1).astype(int)]



# considere un dx de 1 ==> Cell number == Coordinates. Multiplier par Nx,ny pour rendre robuste

In [ ]:
rho_cells

In [ ]:
plt.scatter(np.round(cell_centers[:,0] - 1).astype(int),np.round(cell_centers[:,1] - 1).astype(int),10,rho_cells,norm=colors.LogNorm())
plt.title('Centroids')
plt.ylabel('Y')
plt.xlabel('X')
plt.colorbar()

# Ici le fun begins - Frequency Domain

In [ ]:
#Je crée un modèle de résistivité, sous forme d'un array
res_map = np.empty((30,200),dtype='float')
res_map[0:10,:] = 200
res_map[10:20,:] = 20
res_map[20:30,:] = 100

In [ ]:
plt.imshow(res_map)
plt.xlabel('x')
plt.ylabel('Depth')

In [ ]:
#Where to to the 1D model
position_x = 98
#Noise to be applied on the data
noiseEM = 1  # absolute (per cent of primary signal)

#Nombre de couche dans le modèle foward
nlay = 40
thickness = 1

thk = pg.Vector(nlay - 1, thickness)  # 1m thickness each
centroids = np.cumsum(thk)-thk/2 #Cacul des positions des centres des couches

#On sample dans le model "res_map" la résistivité correspondant aux position de couches. 
res_EM = np.ones(nlay) *  res_map[0,position_x]
for i in centroids:
    if i < 30:
        ab = 29 - np.round((i-0.01)).astype(int) #index dans le modèle resmap
        ab2 = np.floor(i/thickness).astype(int) #index dans le modèle EM
        res_EM[ab2] = res_map[ab,position_x]
        
model = pg.cat(thk, res_EM)  # paste together to one model. 

noiseEM = 1

plt.plot(np.array(res_EM)[:-1],np.array(centroids))


In [ ]:
model.array()
#La logique c'est qu'il y a d'abord toutes les épaisseurs des courches
#couche1, couche2, couche3,...... et ensuite les résistivités. La dernière couche est considérée infine 

In [ ]:
#On définit les propriétés du forward. Espacement entre les coil, et le nombre de frequences. Chaque fréquence == une mesure, et donc une profondeur d'inverstigation différente.
coilspacing = 10
nf = 10
freq = pg.Vector(nf)
freq[0] = 100
freq[1] = 400
freq[2] = 600
freq[3] = 1000
freq[4] = 5000
freq[5] = 8000
freq[6] = 9000
freq[7] = 10000
freq[8] = 20000
freq[9] = 30000

#on crée une fonction forward
fEM = pg.core.FDEM1dModelling(nlay, freq, coilspacing)
#on l'applique au model
dataEM = fEM(model)

#on peux ajouter du bruit. ici désactivé
for i in range(len(dataEM)):
    dataEM[i] += np.random.random(1)[0] * 0

In [ ]:
#on affiches les datas
plt.semilogy(dataEM(0, nf), freq, 'bx', label='syn In-Phase')
plt.semilogy(dataEM(nf, nf*2), freq, 'bo', label='syn Out Phase')
plt.ylim((min(freq), max(freq)))
plt.xlabel("Phase in %")
plt.ylabel("$f$ in Hz")
plt.grid(which='both')
plt.legend(loc="best")

In [ ]:
#Une autre méthode pour comparer. Ca a l'air de donner les memes datas
Data_box_FEM = pg.physics.em.FDEM(x=[1],freqs=freq,coilSpacing=coilspacing)
FOP = pg.core.FDEM1dRhoModelling(centroids, Data_box_FEM.freq(), Data_box_FEM.coilSpacing,-Data_box_FEM.height)
dataEM = FOP(res_EM)


In [ ]:
plt.semilogy(dataEM(0, nf), freq, 'bx', label='syn In-Phase')
plt.semilogy(dataEM(nf, nf*2), freq, 'bo', label='syn Out Phase')
plt.ylim((min(freq), max(freq)))
plt.xlabel("Phase in %")
plt.ylabel("$f$ in Hz")
plt.grid(which='both')
plt.legend(loc="best")

In [ ]:
#affichage des données
In_phase = dataEM(0, nf)
print('InPhase data : ')
print(In_phase)
Out_phase = dataEM(nf, nf*2)
print('OutPhase data : ')
print(Out_phase)

## Inversion FEM
Mainteant on va essayer de retrouver notre modèle initial à partir des données

In [ ]:
#Paramètres de l'inversion
#Pas de limite sur les Rho Apparents ou les épaisseurs
transRhoa = pg.trans.TransLog()
transThk = pg.trans.TransLog()
#limite sur les résisitivés - entre 1 et 1000 ohmm
transRes = pg.trans.TransLogLU(1., 1000.)
transEM = pg.trans.Trans()

In [ ]:
#Nombre de layer du modèle inverse
nLayer_inv = 20
#Modèle initial, avant inversion. 2m de couches, uniformes à 10 ohmm
thk_inv = pg.Vector(nLayer_inv - 1, 2)  # 2m thickness each
res_inv = np.ones(nLayer_inv) *  10
initial_model = pg.cat(thk_inv, res_inv)  # paste together to one model

In [ ]:
fEM_inv = pg.core.FDEM1dModelling(nLayer_inv, freq, coilspacing)
fEM_inv.region(0).setTransModel(transThk)
fEM_inv.region(1).setTransModel(transRes)

lamEM =  100 #regularisation parameters for the inversion

#Toutes les paramètres de l'inverse. Faudrait en parler ca
invEM = pg.core.Inversion(dataEM, fEM_inv, transEM)
invEM.setModel(initial_model)
invEM.setRelativeError(0.1/100)
invEM.setLambda(lamEM)
invEM.setMarquardtScheme(0.8)
invEM.setDeltaPhiAbortPercent(0.00001)
invEM.setMaxIter(40)
invEM.setBlockyModel(True)

modelEM = invEM.run()
respEM = invEM.response()

In [ ]:
np.array(modelEM)

In [ ]:
plt.semilogy(dataEM(0, nf), freq, 'bx', label='syn In-Phase')
plt.semilogy(dataEM(nf, nf*2), freq, 'bo', label='syn Out Phase')

plt.semilogy(respEM(0, nf), freq, 'rx', label='syn In-Phase - Inv')
plt.semilogy(respEM(nf, nf*2), freq, 'ro', label='syn Out Phase - Inv')


plt.ylim((min(freq), max(freq)))
plt.xlabel("Phase in %")
plt.ylabel("$f$ in Hz")
plt.grid(which='both')
plt.legend(loc="best")

In [ ]:
model_Final = np.array(modelEM)
cetr = np.cumsum(model_Final[0:nLayer_inv-2])
res_Final = model_Final[nLayer_inv-1:-2]

Si tu changes les paramètres de l'inversion, ou le modèle initial, la convergence ne sera pas la même est le modèle inverse sera différent. C'est la NON-UNIQUENESS des solutions maggle.

In [ ]:
fig, ax = plt.subplots()

pg.viewer.mpl.drawModel1D(ax, values=res_Final, depths=cetr,plot='semilogx', color='blue',label='Initial Model')
pg.viewer.mpl.drawModel1D(ax, values=res_EM[:-1], depths=centroids,plot='semilogx', color='red',label='Inverted Model')
plt.legend()